In [12]:
import re
import datetime 
from abc import ABC
from markupsafe import Markup
from typing import Dict, Any
import contextlib
from decimal import Decimal
from enum import Enum


class Variable(Enum):
    UNDEFINED = '<undefined>'
    MISSING = '<missing>'
    
    def __str__(self):
        return self.value
    
    def __repr__(self):
        return self.value


def is_defined(value):
    return False if value == Variable.UNDEFINED else True

def is_missing(value):
    return True if value == Variable.MISSING else False

def exist(value):
    return False if value in ['', None, Variable.MISSING, Variable.UNDEFINED] else True

def is_private(string: str) -> bool:
    return True if string.startswith("_") and not string.endswith("_") else False

def unprivate(string: str) -> str:
    return string[ 1: ] if is_private(string) else string

def json_encode(value):
    if value == None:
        return ''
    elif isinstance(value, (str, int, float)):
        return value
    elif isinstance(value, bool):
        return 'true' if value == True else 'false'
    elif isinstance(value, Decimal):
        return str(value)
    elif isinstance(value, list):
        return [json_encode(i) for i in value ]
    elif isinstance(value, dict):
        return {unprivate(k): json_encode(v) for k, v in value.items()}
    elif isinstance(value, Enum):
        return value.value
    elif isinstance(value, tuple):
        try:
            return {k: json_encode(v) for (k,v) in value._asdict().items()}
        except:
            return [json_encode(v) for v in value]
    else:
        try:
            return str(value)
        except BaseException as e:
            print(e)
        finally:
            return value
            

    
class AbstractField(ABC):
    __positional__ = 'required hidden disabled multiple readonly'.split()
    __keyvalue__ = 'placeholder value max min minlength maxlength step pattern'.split()
    __config__ = [*'transform fsize fclass tag default factory'.split(), *__positional__, *__keyvalue__]
    __config_defaults__ = dict(required=True, fsize=(12,6,3), tag='input:text')
    
    def __init__(self, *args, **kwargs) -> None:
        
        if len(args) > 0:
            if type(args[0]) == str:
                self.owner = None
                self.owner_name = args[0]
            else:
                self.owner = args[0]
                self.owner_name = self.owner.__name__
        else:
            self.owner = Variable.UNDEFINED
            self.owner_name = Variable.UNDEFINED
    
        for key, val in kwargs.items():
            if key in self.__config__:
                setattr(self, key, val)
        for key in self.__config__:
            if not hasattr(self, key):
                if key in self.__config_defaults__.keys():
                    setattr(self, key, self.__config_defaults__[key])
                else:
                    setattr(self, key, Variable.UNDEFINED)

    def get_form_attrs(self):
        result = ''
        for k,v in vars(self).items():
            if is_defined(v):
                if k in self.__keyvalue__:
                    result += f' {k}="{v}"'
                elif k in self.__positional__ and v == True:
                    result += f' {k}'
        return result

    def __set_name__(self, obj, name):
        self.public_name = name 
        self.private_name = f'_{name}'

    def pre_validation(self, obj, value):
        if not exist(value):
            if is_defined(self.factory):
                value = self.factory()
            elif is_defined(self.default):
                value = self.default           
        return value
    
    def check_required(self, obj, value):
        if is_defined(self.required):
            if not exist(value):
                raise ValueError(f'{type(obj).__name__}.{self.public_name} cannot be "{value}"')
            
    def post_validation(self, obj, value):
        if exist(value):
            if is_defined(self.transform):
                if not issubclass(type(self), (EnumField, ReferenceField)):
                    return self.transform(value)
        return value
            
            
    def validate(self, obj, value):
        self.check_required(obj, value)
        if exist(value):
            if is_defined(self.min):
                if float(self.min) > value:    
                    raise ValueError(f'{self._name_} of {type(obj).__name__} is "{value}" and cannot be lesser then {self.min}')
            if is_defined(self.max):
                if float(self.max) < value:    
                    raise ValueError(f'{self._name_} of {type(obj).__name__} is "{value}" and cannot be greater then {self.max}')
            if is_defined(self.minlength):
                if float(self.minlength) > len(value):    
                    raise ValueError(f'{self._name_} of {type(obj).__name__} is "{value}" and cannot has length lesser then {self.minlength}')
            if is_defined(self.maxlength):
                if float(self.maxlength) < value:    
                    raise ValueError(f'{self._name_} of {type(obj).__name__} is "{value}" and cannot be greater then {self.maxlength}')
            if is_defined(self.pattern):
                if not re.match(self.pattern, value):
                    raise ValueError(f'{self._name_} of {type(obj).__name__} is "{value}" does not match {self.pattern}')
                    
    def __set__(self, obj, value):
        pre = self.pre_validation(obj, value)
        self.validate(obj, pre)
        post = self.post_validation(obj, pre)
        setattr(obj, self.private_name, post)
        
    def parse(self, obj, value):
        return value 
    
    def __get__(self, obj, owner=None):
        value = getattr(obj, self.private_name)
        return self.parse(obj, value)
    

class  RegularField(AbstractField):
    pass 


class  EnumField(AbstractField):
    pass 

        
class ReferenceField(AbstractField):
    pass 


class AbstractModel(ABC):
    __objname__ = ''
    __table__ = ''
    __singular__ = ''
    __plural__ = ''
    
    def __init__(self, *args, **kwargs) -> None:
        self._args_ = args 
        self._kwargs_ = kwargs
        for k in self.fields().keys():
            if k in self._kwargs_.keys():
                setattr(self, k, kwargs[k])
            else:
                setattr(self, k, None)

    def __extra_fields__(self):
        extra_fields = dict()
        for k,v in self._kwargs_.items():
            if not k in self.fields().keys():
                extra_fields.update({k:v}) 
        return extra_fields
    
    @classmethod
    def __regular_fields__(cls):
        return {k:v for (k,v) in cls.fields().items() if isinstance(v, RegularField)}
    
    @classmethod
    def __reference_fields__(cls):
        return {k:v for (k,v) in cls.fields().items() if isinstance(v, ReferenceField)}
    
    @classmethod
    def __enum_fields__(cls):
        return {k:v for (k,v) in cls.fields().items() if isinstance(v, EnumField)}
    
    def __str_html__(self) -> str:
        return Markup(f'<h3>{str(self)}</h3>')  
        
    def __str__(self) -> str:
        return ', '.join([str(v.__get__(self)) for v in self.fields().values() if exist(v.__get__(self))])
                
    def __repr__(self) -> str:
        return f'{type(self).__name__}({", ".join([f"{k}={str(v.__get__(self))}" for k, v in self.fields().items()])})' 
    
    def __json__(self) -> dict:
        result = {}
        for k,v in self.fields().items():
            result[k] = json_encode(v.__get__(self))
        return result
    
    @property
    def json(self) -> dict:
        return self.__json__()
    
    @classmethod
    def fields(cls):
        return {k:v for (k,v) in vars(cls).items() if isinstance(v, AbstractField)}

    def defaults(self):
        defaults = {}
        for k,v in self.fields().items():
            if is_defined(v.factory):
                defaults.update({k: v.factory()})  
            elif is_defined(v.default):
                defaults.update({k: v.default})
        return defaults


class AbstractManager(ABC):
    
    @classmethod 
    def get_key(cls, instance: AbstractModel):
        return getattr(instance, 'key', None)
            
    @classmethod 
    def get_meta(cls, instance: AbstractModel, owner: str):
        return {
            'model':type(instance).__name__,
            'created': datetime.date.today(),
            'owner': owner
        }
    
    @classmethod 
    def parse_to_db(cls, instance: AbstractModel):
        key = cls.get_key(instance)
        data = instance.json
        model = type(instance).__name__
        
        


ModuleNotFoundError: No module named 'deta'

# Elizete Felix Teodoro Mendes
 
    DATA: 29 NOV 2021.
    MOTIVO: Relatório Médico Psiquiátrico para Cirurgia Bariátrica.
    
--- 

Paciente em acompanhamento há mais de 7 anos com apresentação inicial de quadro hipomaníaco e história prévia de episódios depressivos recorrentes. Fez uso prévio de medicações antidepressivas sem boa resposta e hoje esta em uso de lítio 600mg + lamotrigina 200mg com excelente controle do humor. Não tem exibido crises de euforia ou depressão há vários anos e apesar da estabilidade não tem conseguido controlar o peso, em parte pelas medicações estabilizadoras utilizadas. 
A paciente sempre foi preocupada com o excesso ponderal, tendo sido inclusive o principal motivo para busca de suporte médico ao longo da vida antes das crises afetivas. Ao longo do tratamento ocorreu ganho ponderal progressivo de 70kg (2014) para 89kg (atual). O excesso de peso compromete a auto estima da paciente e esta associado a comorbidades diversas como hipertrigliceridemia, hipercolesterolemia, esteatose hepática, hiperuricemia e pré-diabetes. A paciente não é usuária de substâncias ilícitas e não apresenta padrão de abuso/dependência de uso de álcool. Nunca apresentou sintomas psicóticos primários e é totalmente lúcida e capaz de compreender todas as questões relacionadas à cirurgia bariátrica como riscos, benefícios e necessidades secundários ao procedimento. Apesar da necessidade de perda de peso a paciente apresenta restrição para uso de medicação antiobesidade. Não há entretanto qualquer limitação do ponto de vista psiquiátrico para a abordagem cirúgica, com expectativa de melhora do quadro psiquiátrico por questões clínicas (redução da inflamação sistêmica e do distúrbio metabólico) e psicológicas (auto estima). 

*Ao exame: humor eutímico, congruente com afeto; pensamento sem alterações na forma, conteúdo e fluxo; ausência de atividade delirante ou alucinatória; pragmatismo preservado; insight presente; juízo crítico da realidade presente.* 

A paciente encontra-se estável do ponto de vista afetivo, sem sintomas importantes há vários anos. A obesidade tem piorado ao longo do tempo, sem controle mesmo após uso de estratégias diversas (liraglutida, orlistat, topiramato, bupropiona, naltrexone), comprometendo a saúde mental e geral da paciente. Do ponto de vista psiquiátrico não há restrição para cirurgia bariátrica. As medicações atuais podem ser administradas em formulação orodispersível (lamotrigina) ou dissolvidas (litio) e poderiam ser interrompidas/reduzidas temporiamente caso seu uso seja contraindicado durante a cirurgia. 

- F31.7 - Transtorno afetivo bipolar, atualmente em remissão

Me encontro à disposição para esclarecimentos adicionais caso necessário. 

Atenciosamente,

In [27]:
class Person(AbstractModel):
    fullname = AbstractField()

In [28]:
p = Person(fullname='Daniel Victor')

In [29]:
p

Person(fullname=Daniel Victor)

In [30]:
x = AbstractManager.parse_to_db(p)

In [31]:
x